In [1]:
import flowermd # wrapper we use on top of hoomd
import time # for measuring simulation time
import gsd # to work with simulation data files
import gsd.hoomd # to work with simulation data files directly coming from hoomd sims
import hoomd # MD simulation engine used for initializing and running sims
import mbuild as mb # library used to build Molecule objects
import numpy as np # common library used for a plethora of things, such as indexing, numeric constants, linear algebra, etc.
import warnings # used to provide the user warnings rather than errors
from flowermd.base import Pack, Simulation, System, Molecule # importing useful functions from flowermd to simplify initializing and running 
from flowermd.library import LJChain # lennard jones polymer chains with scalable length
from flowermd.library.forcefields import BeadSpring # forcefield template used in order to define interactions within sim
from flowermd.utils import get_target_box_number_density # used to define final density to shrink box to in order to minimize space
from mbuild.compound import Compound # base compound object to create molecule geometry
from mbuild.lattice import Lattice # lattice object to initialize lattice spacing and points
import unyt as u # module used for unit definitions
warnings.filterwarnings('ignore') # ignore warnings

/Users/anahumr/miniconda3/envs/mxene/lib/python3.12/site-packages/gmso/core/element.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


## Define Flake Geometry

In [2]:
class Flake(System):
    def __init__(
        self,
        x_repeat,
        y_repeat,
        n_layers,
        base_units=dict(),
        periodicity=(True, True, False),
    ):
        surface = mb.Compound(periodicity=periodicity)
        a = 3**.5
        lattice = Lattice(
            lattice_spacing=[a,a,a],
            lattice_vectors=  [[a,0,0],[a/2,3/2,0],[0,0,1]],
            lattice_points={"A": [[1/3,1/3,0], [2/3, 2/3, 0]]},
        ) # define lattice vectors, points, and spacings for flakes
        Flakium = Compound(name="F", element="F") # defines an atom that will be used to populate lattice points
        layers = lattice.populate(
            compound_dict={"A": Flakium}, x=x_repeat, y=y_repeat, z=n_layers
        ) # populates the lattice using the previously defined atom for every "A" site, repeated in all x,y, and z directions
        surface.add(layers) # adds populated flake lattice layers to the 'surface' compound, which represents our flake structure 
        surface.freud_generate_bonds("F", "F", dmin=0.9, dmax=1.1) # generates bonds depending on input distance range, scales with lattice
        surface_mol = Molecule(num_mols=1, compound=surface) # wraps into a Molecule object, creating "1" instance of this molecule

        super(Flake, self).__init__(
            molecules=[surface_mol],
            base_units=base_units,
        )

    def _build_system(self):
        return self.all_molecules[0]

## Define Forcefield for WCA Interactions
Essentially zero attraction; pure repulsion.

In [3]:
ff = BeadSpring(
    r_cut=2**(1/6),  # r_cut value defines the radius in which a given particle will interact with another.
    beads={
        "A": dict(epsilon=1.0, sigma=1.0),  # chains, epsilon = well depth, defines strength of attractive forces between two molecules
        "F": dict(epsilon=1.0, sigma=1.0),  # flakes, sigma = distance between two particles where PE is zero
    },
    bonds={
        "F-F": dict(r0=1.0, k=1000),
        "A-A": dict(r0=1.0, k=1000.0),  # r0 = equilibrium distance of bonded particles, k = stiffness constant
    },
    angles={
        "A-A-A": dict(t0=2* np.pi / 3., k=100.0),   
        "F-F-F": dict(t0=2 * np.pi / 3., k=5000),
    },
    dihedrals={
        "A-A-A-A": dict(phi0=0.0, k=0, d=-1, n=2), # do not worry about dihedrals
        "F-F-F-F": dict(phi0=0.0, k=500, d=-1, n=2),
    }
)

# Define Simulation

In [4]:
def run_simulation(N_chains=20, initial_dens=0.001, final_dens=0.3, N_flakes=5, 
                   chain_length=10, dt=0.005, temp=3, device='CPU',
                   write_frequency=1000, shrink_steps=5e5, steps=1e6):
    """
    Runs an entropy-driven aggregation simulation on HOOMD with the specified parameters.

    Parameters:
    - N_chains: Number of polymer chains.
    - initial_dens: Initial density of the system.
    - final_dens: Final target density for the system.
    - N_flakes: Number of flakes.
    - chain_length: Length of each chain.
    - dt: Time step size.
    - temp: Temperature.
    - device: 'CPU' or 'GPU' for the simulation device.
    - write_frequency: Write frequency of simulation.
    - shrink_step: Number of steps to run shrink.
    - steps: Number of steps simulation runs.
    """
    
    # For command line testing - added to check that args are parsed correctly
    print(
        f"Running simulation with {N_chains} chains, "
        f"initial density {initial_dens}, final density {final_dens}, "
        f"{N_flakes} flakes, chain length {chain_length}, "
        f"dt {dt}, temperature {temp}, device {device}, "
        f"write frequency {write_frequency}, {shrink_steps} shrink_steps, "
        f"and {steps} steps."
    )

    # Set device (CPU or GPU), defaulted to CPU
    if device == 'GPU':
        device = hoomd.device.GPU()
    else:
        device = hoomd.device.CPU()

    # Initialize the chain and sheet
    kg_chain = LJChain(lengths=chain_length, num_mols=N_chains)
    sheet = Flake(x_repeat=5, y_repeat=5, n_layers=1, periodicity=(False, False, False))
    
    # Initialize the system
    system = Pack(molecules=[Molecule(compound=sheet.all_molecules[0], num_mols=N_flakes), kg_chain], 
                  density=initial_dens, packing_expand_factor=6, seed=2)
    
    # Get the target box size based on the final density
    target_box = get_target_box_number_density(density=final_dens * u.Unit("nm**-3"), n_beads=(500 + (N_chains * 10)))

    # Prepare file output
    gsd = f"{N_chains}_{chain_length}mer{N_flakes}f_{dt}dt.gsd" # Name of output gsd files
    log = f"{N_chains}_{chain_length}mer{N_flakes}f_{dt}dt.txt" # Name of output log files
    start_file = f"{N_chains}_{chain_length}mer{N_flakes}f_{dt}dt_start.txt" # Name of output start of sim

    # Saving simulation starting point
    sim_start = int(shrink_steps/write_frequency)

    with open(start_file, "w") as f:
        f.write(str(sim_start))

    # Initialize the simulation object
    sim = Simulation(initial_state=system.hoomd_snapshot, forcefield=ff.hoomd_forces, device=device, dt = dt, 
                 gsd_write_freq=int(write_frequency), log_file_name = log, gsd_file_name = gsd)
    start_shrink = time.time()

    # Run the simulation with volume update and thermalization
    sim.run_update_volume(final_box_lengths=target_box, kT=6.0, n_steps=shrink_steps,tau_kt=100*sim.dt,period=10,thermalize_particles=True) # shrink simulation run
    end_shrink = time.time()
    start_run = time.time()
    sim.run_NVT(n_steps=steps, kT=temp, tau_kt=dt*100) # simulation run
    end_run = time.time()

    sim.flush_writers() # Flush data into output files
    del sim # Drop references so files are closed

    # Visualize results - TODO, add as function parameters or prompt options after simulation
    !mv "{log}" "{gsd}" "{start_file}" ../analysis/ # places created files in the analysis folder
    # Output simulation time stats
    print(f"Shrink phase completed in {(end_shrink - start_shrink)} seconds")
    print(f"Run phase completed in {(end_run - start_run)} seconds")
    print(f"Total time: {((end_run - start_run)+(end_shrink - start_shrink))/60:.2f} minutes")

## Function Usage:
```run_simulation(N_chains, initial_dens, final_dens, N_flakes,``` \
```chain_length, dt, temp, device['CPU' or 'GPU'],  write_frequency,```\
```shrink_steps, steps)```
## Run function below:
**The function call below MUST be commented out if you are using the command line.** Otherwise, this particular call will run in addition to your specified experiment. 

In [5]:
# Example function usage
# run_simulation(20, 0.001, 0.3, 5, 10, 0.005, 3, 'CPU', 1000, 5e5, 1e6);

# TODO: relocate in_notebook() definition

# Command Line Support

In [6]:
# Add command line usage
def in_notebook():
    try:
        from IPython import get_ipython
        return get_ipython() is not None
    except ImportError:
        return False
        
if __name__ == "__main__" and not in_notebook():
    import argparse
    import textwrap

    # Help/usage
    parser = argparse.ArgumentParser(
        description="Run HOOMD simulation with custom parameters.",
        formatter_class=argparse.RawDescriptionHelpFormatter,
        epilog=textwrap.dedent('''
            Example:
              python clean_sim.py --N_chains 10 --temp 2.5 --device CPU --shrink_steps 5e2 --steps 1e4
        ''')
    )

    parser.add_argument("--N_chains", type=int, default=30, help="Number of polymer chains (default: 30)")
    parser.add_argument("--initial_dens", type=float, default=0.001, help="Initial density (default: 0.001)")
    parser.add_argument("--final_dens", type=float, default=0.3, help="Final density (default: 0.3)")
    parser.add_argument("--N_flakes", type=int, default=10, help="Number of flakes (default: 10)")
    parser.add_argument("--chain_length", type=int, default=10, help="Length of each chain (default: 10)")
    parser.add_argument("--dt", type=float, default=0.005, help="Time step (default: 0.005)")
    parser.add_argument("--temp", type=float, default=3, help="Simulation temperature (default: 3)")
    parser.add_argument("--device", choices=["CPU", "GPU"], default="CPU", help="Computation device (default: CPU)")
    parser.add_argument("--write_frequency", type=float, default=1000, help="Write frequency (default: 1000)")
    parser.add_argument("--shrink_steps", type=float, default=5e5, help="Duration of shrinkage in steps (default: 5e5)")
    parser.add_argument("--steps", type=float, default=1e6, help="Total duration of simulation in steps (default: 1e6)")
    
    args = parser.parse_args()

    run_simulation(
        N_chains=args.N_chains,
        initial_dens=args.initial_dens,
        final_dens=args.final_dens,
        N_flakes=args.N_flakes,
        chain_length=args.chain_length,
        dt=args.dt,
        temp=args.temp,
        device=args.device,
        write_frequency=args.write_frequency,
        shrink_steps=args.shrink_steps,
        steps=args.steps
    )

# TODO: command line cannot access !mv "{log}" "{gsd}" "{start_file}" usage. We will need to change to subprocess or os.system

## Notes on CLI:
At present, this command line support is built for .py files, and you must use\
```jupytext --to py clean_sim.ipynb```
to transform the file before running.

After that, you can run the command (for example):\
```python clean_sim.py --N_chains 10 --temp 2.5 --device CPU --shrink_steps 5e5 --steps 1e6```.